In [1]:
import os, sys, subprocess, re, requests, json, warnings
import numpy as np
import pandas as pd
from urllib.request import urlopen
from time import sleep

warnings.filterwarnings('ignore')

# # if necessary
# from Bio import SeqIO
# from Bio.PDB.DSSP import DSSP
# from Bio.PDB.DSSP import PDBParser
# import biotite.application.dssp as dssp

In [2]:
def get_url(url, **kwargs):
    '''
    Obatin a response from a given url
    '''
    response = requests.get(url, **kwargs);

    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()

    return response

In [3]:
# uniprot API URL
WEBSITE_API = "https://rest.uniprot.org/uniprotkb/"

# minimum length of a helix to be considered
helix_min_len = 6

In [4]:
def find_SS_location(res_seq, ss_seq, ss_symbol):
    '''
    Return residues that belong to helix
    as a list of residues and their locations
    Note that '*' is added in the end of both AA and prediction strings,
    thus precition[0-1] and predition[last] return non-H anyway,
    preventing the bug when predciton starts or ends with H
    '''
    helix_res_loc_dict = dict()
    
    res_seq += '*'
    ss_seq += '*'

    for i in range(len(ss_seq)):
        if (ss_seq[i-1] != ss_symbol) & (ss_seq[i] == ss_symbol):
            helix_start = i

        elif (ss_seq[i-1] == ss_symbol) & (ss_seq[i] != ss_symbol):
            helix_end = i
            
            # pack helix residues and locations into a dictionary
            helix_res = res_seq[helix_start:helix_end]
            # helix loc as a string
#             helix_loc = str(helix_start+1) + '-' + str(helix_end) # residue number is non-pythonic and starts with 1, not 0
            # helix loc as a list
            helix_loc = list()
            helix_loc.append(helix_start+1)
            helix_loc.append(helix_end) # residue number is non-pythonic and starts with 1, not 0
            
            helix_res_loc_dict[helix_res] = helix_loc

    return helix_res_loc_dict

# 1. FASTA sequence

In [15]:
# protein list import
df = pd.read_excel('./SourceData/AH_protein_list.xlsx')
columns = df.iloc[0, 1:]
df = df.iloc[1:, 1:]
df.columns = columns
df = df.reset_index()
df = df.drop(['index'], axis=1)

In [20]:
# Protein entry list
entry_list = df.Entry.unique().tolist()

In [8]:
# Obtain FASTA seq from Uniprot, make AH position data
for entry in entry_list:
    print(entry)
    try:
        r = get_url(f'{WEBSITE_API}search?query=accession:{entry}&format=fasta&compressed=false')
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
        break
        
    # export fasta
    fasta = r.text
    with open('./TrainingData/Original_fasta/%s.fas' % entry, 'w') as f:
        f.write(fasta)
        f.close()
        
    sleep(1)

P49585
P37840
P02647
P32567
Q12164
Q12164
P71021
Q7KLE5
O35179
P11006
P37817
Q9Y6I3
O00499
Q9UBW5
P84080
P01501
B6GZG8
P22943
P03588
Q9VC57
Q12402
O04616
P49790
Q503I3
Q04359


# s4pred for ss prediction, then DeepTMHMM for subtracting TM locations

In [9]:
# s4pred
model = '../s4pred/run_model.py'

# DeepTMHMM
import biolib
deeptmhmm = biolib.load('DTU/DeepTMHMM')

2022-08-15 19:21:18,087 | INFO : Loaded project DTU/DeepTMHMM:1.0.12


In [11]:
# make a home df for output
df_output = pd.DataFrame(columns=['Entry','Gene_name','Helix_seq','Helix_loc'])

In [17]:
df.head()

,Entry,Organism,Gene_name,AH_location_start,AH_location_end,Seq,Source,Helix in AF?,Helix in s4pred
0,Q9UH99,Human,SUN2,155,190,NaN,NaN,NaN,NaN
1,Q8N6T3,NaN,ARFGAP1,199,223,NaN,NaN,NaN,NaN
2,Q15643,NaN,GMAP-210,1,38,NaN,NaN,No,NaN
3,P20606,NaN,Sar1p,1,23,NaN,NaN,NaN,NaN
4,Q14534,NaN,SQLE,61,71,SQFALFSDILS,"Chua, N.K., V. Howe, N. Jatana, L. Thukral, an...",No,Yes


In [19]:
for i, entry in enumerate(entry_list):
    gene_name = df.iloc[i, 2]
    print(entry, gene_name)
    
    # run s4pred from command line
    command = \
    'python %s --outfmt fas ./TrainingData/Original_fasta/%s.fas > ./TrainingData/s4pred_output/%s_ss.fas' \
    % (model, entry, entry)
    subprocess.run(command, shell=True)
    print(entry, ': s4pred done')
    
    # obtain AA seq and SS prediction results as string
    file = open(f'./TrainingData/s4pred_output/{entry}_ss.fas', 'r')
    lines = file.readlines()
    aa_seq = lines[-2].strip()
    sse = lines[-1].strip()
    
    try:
        # find TM by deepTMHMM
        ## Run DeepTMHMM
        deeptmhmm_res = deeptmhmm.cli(args=f'--fasta ./TrainingData/Original_fasta/{entry}.fas')
        ## Save the results
        deeptmhmm_res.save_files(f"./TrainingData/DeepTMHMM_output/{entry}/")

        # read and store TM location prediction
        deeptmhmm_results = open('./TrainingData/DeepTMHMM_output/Q9UH99/deeptmhmm_results.md')
        lines = deeptmhmm_results.readlines()
        tm = lines[8]
    except:
        print('deepTMHMM failed')
        break
    
    # replace H by M in sse where TM is predicted
    sse = list(sse) # switch to a list to make it mutable
    print(len(sse), len(tm))
    for i in range(len(sse)):
        if tm[i] == 'M':
            sse[i] = 'M'
    sse = ''.join(sse) # recover it to a string
    
    # find locations of H
    helix_dict = find_SS_location(aa_seq, sse, 'H')
    # remove short H chains
    for k in list(helix_dict):
        if (helix_dict[k][1] - helix_dict[k][0] + 1) < helix_min_len:
            del helix_dict[k]
    
    # find locations of TM
    tm_dict = find_SS_location(aa_seq, tm, 'M')
    
    # make a _df, then append it to the home df_output
    _df = pd.DataFrame()
    ## For non-TM helix
    _df['Helix_seq'] = helix_dict.keys()
    _df['Helix_loc'] = helix_dict.values()
    _df['TM?'] = 0
    
    ## For non-TM helix
    _df['Helix_seq'] = tm_dict.keys()
    _df['Helix_loc'] = tm_dict.values()
    _df['TM?'] = 1
    
    ## Entry and gene name
    _df['Entry'] = entry
    _df['Gene_name'] = gene_name
    df_output = df_output.append(_df)
    
    sleep(1)

Q8N6T3 SUN2
Q8N6T3 : s4pred done
2022-08-15 19:47:31,602 | INFO : Job "5316541a-0d4a-48c4-a99d-99910f74f405" is starting...
2022-08-15 19:47:52,759 | INFO : Cloud: Server capacity is being allocated. Please wait...
2022-08-15 19:48:03,042 | INFO : Cloud: Server capacity is being allocated. Please wait...
2022-08-15 19:48:13,323 | INFO : Cloud: Server capacity is being allocated. Please wait...
2022-08-15 19:48:23,569 | INFO : Cloud: Server capacity is being allocated. Please wait...
2022-08-15 19:48:33,807 | INFO : Cloud: Server capacity is being allocated. Please wait...
deepTMHMM failed


In [18]:
# iterator = SeqIO.read(file, 'fasta')
# records = SeqIO.to_dict(iterator)
# print(iterator.seq)
# # print(list)
# #     print(seq_rec.seq)
# file.close()
# print(sse)

# 2. Helix sequence extracted from AlphaFold predicted struture

## DSSP - primarily used

In [ ]:
file_path = '../../../AF-Q9WU40-F1-model_v3.pdb'

In [31]:
p = PDBParser()

In [70]:
def get_aaSeq_and_ss(pdb_filepath):

    # empty string for output    
    res_seq = ''
    ss_seq = ''
    
    # get dssp from the PDB file
    strcuture = p.get_structure('name', pdb_filepath)
    model = strcuture[0]
    dssp = DSSP(model, pdb_filepath)
    
    # parse dssp for residue symbol and ss info
    for i in range(len(dssp.keys())): # len of the protein
        # residue key
        a_key = list(dssp.keys())[i]

        # residue symbol and SS
        res = list(dssp[a_key])[1]
        ss = list(dssp[a_key])[2]
        res_seq += res
        ss_seq += ss

    return res_seq, ss_seq

In [71]:
res_seq, ss_seq = get_aaSeq_and_ss(file_path)

In [85]:
find_SS_location(res_seq, ss_seq, ss_symbol='H')

{'DEELFSQLRR': '15-24',
 'RPVYLKKLKKLREEEQQQQQQQQQQQHRA': '37-65',
 'EEELLQQFKRE': '461-471',
 'SFSAHYLSMFLLTAACLFFLILGLTYLGMR': '478-507',
 'ESEKNLLMSTLYKLHDRLAQIAGDHEC': '531-557',
 'VQEAAAYLKNL': '567-577',
 'EDVFNTSLLWIFKN': '582-595',
 'FWCRFRRAFITVTHRLLLLCLGVVLVCVALRYMRYRWTKEEEETRQMYDMVVKIIDVLRSHNEACQE': '630-696',
 'LPHVRDSL': '706-713',
 'KKVWDRAVDFLAAN': '723-736',
 'WHLAIQEAILEK': '812-823',
 'PEYAGKAFKAL': '851-861',
 'LDRYHHR': '878-884',
 'SHLR': '906-909'}

## Using biotite - but not used

In [8]:
# from tempfile import gettempdir, NamedTemporaryFile
# import biotite.structure as struc
# import biotite.structure.io as strucio
# import biotite.structure.io.pdb as pdb
# import biotite.database.rcsb as rcsb

# file_path = rcsb.fetch("1l2y", "pdb", gettempdir())
# pdb_file = pdb.PDBFile.read(file_path)
# tc5b = pdb_file.get_structure()
# print(type(tc5b).__name__)
# # print(tc5b.stack_depth())
# print(tc5b.array_length())
# print(tc5b.shape)

# array = strucio.load_structure(file_path)
# sse = struc.annotate_sse(array, chain_id='A')

# sse = ''.join(sse)

In [ ]:
dssp = DSSP(model, '/local-pdb/1mot.pdb', dssp='mkdssp')

# DeepTMHMM

In [ ]:
import biolib

deeptmhmm = biolib.load('DTU/DeepTMHMM')

# Run DeepTMHMM
deeptmhmm_res = deeptmhmm.cli(args='--fasta ./TrainingData/Original_fasta/Q15643.fas')

# # Save the results
# deeptmhmm_res.save_files("./biolib_results/")

# deeptmhmm_res.ipython_markdown()

2022-08-14 13:54:35,662 | INFO : Loaded project DTU/DeepTMHMM:1.0.12
2022-08-14 13:54:36,354 | INFO : Job "330cb727-5aa1-49ff-9815-d420f961165c" is starting...
2022-08-14 13:54:39,369 | INFO : Started compute node
2022-08-14 13:54:39,985 | INFO : Compute Node: Initializing
2022-08-14 13:54:40,591 | INFO : Job "330cb727-5aa1-49ff-9815-d420f961165c" running...
2022-08-14 13:54:41,494 | INFO : Compute Node: Pulling images...
